In [90]:
import os
import sys
import glob
import json
import requests
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
# from google.cloud import bigquery
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

In [101]:
proj_id = 'nlp-project-253707'
table_id = 'news_dataset.news'

In [102]:
sql = '''
SELECT url
FROM `nlp-project-253707.news_dataset.news`
'''

tagged_news = gbq.read_gbq(sql, project_id=proj_id)
tagged_news_url = list(tagged_news.url)

In [4]:
fnames = glob.glob('newsdata/downloaded/*.json')

In [104]:
n_tag = 500
urls = []
texts = []

n = 0
for fname in fnames:
    js = json.loads(Path(fname).read_text())
    
    if js['url'] not in tagged_news_url:        
        texts.append(js['text'])
        urls.append(js['url'])
        n += 1
        print('\r{}'.format(n), end='')
        if n == n_tag: break

500

In [ ]:
apikey = {
            'gem763': 'YbEhDr9hAYmd', 
    'projectester01': 'PUcCZkyO5wPs',
}

In [107]:
# headers = {"Authorization":"Token YbEhDr9hAYmd"}
headers = {"Authorization":"Token PUcCZkyO5wPs"}
endpoint = 'https://api.uclassify.com/v1/uclassify/IAB Taxonomy v2/classify'
data = json.dumps({'texts': texts})

response = requests.post(endpoint, headers=headers, data=data)

In [108]:
records = []
updated_at = str(pd.Timestamp.utcnow().date())

for i, resp in enumerate(tqdm(response.json())):
    cl = max(resp['classification'], key=lambda cand:cand['p'])    
    records.append({
        'url': urls[i], 
        'text': texts[i], 
        'uclass': cl['className'].split('_')[0], 
        'prob': cl['p'], 
        'updated_at': updated_at, 
    })
    
df = pd.DataFrame(records); df

,prob,text,uclass,updated_at,url
0,0.964727,"IOWA CITY, Iowa (AP) - The University of Iowa ...",music and audio,2019-09-25,https://washingtontimes.com/news/2019/sep/16/i...
1,0.696007,\n\nAn alarming share of future federal revenu...,personal finance,2019-09-25,https://washingtonpost.com/outlook/congress-is...
2,0.999986,"During his Twitter tirade, Brown vented about ...",sports,2019-09-25,https://theatlantic.com/ideas/archive/2019/09/...
3,0.429576,Photo: Axelle/Bauer-Griffin/FilmMagic\n\nLast ...,music and audio,2019-09-25,https://vulture.com/2019/09/james-corden-slams...
4,0.782219,Why have China and Russia — America's two bigg...,business and finance,2019-09-25,https://uk.investing.com/analysis/chart-of-the...
5,0.999601,"\n\nWASHINGTON, DC - NOVEMBER 29: Passengers w...",travel,2019-09-25,https://washingtonpost.com/transportation/2019...
6,0.547811,"Ware2Go, an independent subsidiary of UPS Inc....",business and finance,2019-09-25,https://finance.yahoo.com/news/ware2go-adds-di...
7,0.390820,"HIKO, Nev. (AP) - The festivals are over and E...",events and attractions,2019-09-25,https://washingtontimes.com/news/2019/sep/22/a...
8,0.892190,The TAKE with Rick Klein\n\nInterested in The ...,news and politics,2019-09-25,https://abcnews.go.com/Politics/note-rivals-te...
9,0.943290,A primary challenger to Rep. Alexandria Ocasio...,news and politics,2019-09-25,https://foxnews.com/media/ocasio-cortez-challe...


In [109]:
gbq.to_gbq(df, table_id, project_id=proj_id, if_exists='append')

1it [00:08,  8.27s/it]
